In [1]:
import numpy as np
import pickle as pkl
import os
import time
import cv2
import itertools
import dlib
import openface
from sklearn.metrics import v_measure_score as vms
from sklearn.metrics import adjusted_rand_score as ars
from sklearn.metrics import fowlkes_mallows_score as fms
from sklearn.metrics import adjusted_mutual_info_score as amis
from sklearn.metrics import homogeneity_score as hs
from sklearn.metrics import completeness_score as cs
from scipy import misc
from scipy.optimize import linear_sum_assignment as lsa
#import self made modules
import GMM_updation_uni as GMMu

In [2]:
def overlap_in_percent(ref, new):
	lft = max(new[0]-new[2]/2, ref[0]-ref[2]/2)
	rt = min(new[0]+new[2]/2, ref[0]+ref[2]/2)
	top = max(new[1]-new[3]/2, ref[1]-ref[3]/2)
	bot = min(new[1]+new[3]/2, ref[1]+ref[3]/2)
	if rt-lft < 0 or bot-top < 0:
		return 0.0
	else:
		return 100.0 * (rt-lft)*(bot-top) / (0.0001 + max(new[2]*new[3], ref[2]*ref[3]))

def euc_dist_sq(x1,x2):
	return np.sum((x1-x2)**2)

def get_vid_list(feats_list):
	with open("./resources/file_Names_nottinghill.txt","r") as fp:
		lsX = pkl.load(fp)
	print("No. of faces = " + str(len(lsX)) )
	np_filedata = np.zeros((len(lsX), 3), dtype=int)
	for i in range(len(lsX)):
		strname = lsX[i]
		#print(strname)
		np_filedata[i,0] = int(strname[0:4]) #Shot IDs
		np_filedata[i,1] = int(strname[4:11]) #Frame IDs for each shot
		np_filedata[i,2] = int(strname[11:13]) #person IDs for each frame(different for each frame)
	np_cords = np.loadtxt('./resources/loc_info_nottinghill.txt', dtype=float)
	np_labels = np.loadtxt('./resources/labels_nottinghill.txt', dtype=int)
	# for every frame, this array has a row in form [xcentre ycentre width height] 
	j_c=0
	track_cnt=0

	ls_all = []
	ls_mats = []
	ls_all_labels = []
	ls_all_feats = []
	print("ss range: "+str(np_filedata[-1,:]))
	for ss in range(0,np_filedata[-1,0]+1):

		fno = 0 #frame no. of 'ss'th shot
		ls_tracks = []
		ls_pos = []
		ls_labels = []
		ls_feats = []
		np_tr = np.zeros((len(ls_tracks),len(ls_tracks)))
		print("shot: " + str(ss) + ", j_c: " + str(j_c))

		while (np_filedata[j_c,0] == ss):
			#ss is the shot number and j is the overall number
			data_file = []
			data_loc = []
			data_label = []
			data_feat = []
			jj = j_c #for getting faces in this frame
			while(np_filedata[j_c,1] == np_filedata[jj,1]):
				data_file.append(lsX[j_c])
				data_loc.append(np_cords[j_c,:])
				data_label.append(np_labels[j_c])
				data_feat.append(feats_list[j_c])
				j_c+=1
				if j_c >= len(lsX):
					break
			ll_matrix = np.zeros((len(ls_pos),len(data_loc)),dtype=float)
			dd_matrix = np.zeros((len(ls_pos),len(data_loc)),dtype=float)
				#fill ll_matrix
			for j in range(0,len(ls_pos)):		#ls no.
				for k in range(0,len(data_loc)): 	#data no
					ll_matrix[j,k] = overlap_in_percent(ls_pos[j], data_loc[k])
					dd_matrix[j,k] = 4.0-euc_dist_sq(ls_feats[j][-1], data_feat[k])

			Threshold = 85
			dd_Threshold = 3.0
			#print("ll_mat:\n"+str(ll_matrix))
			data_ind = np.arange(len(data_loc))
			tr_ind = np.arange(len(ls_pos))
			ls_diff = []

			while (data_ind.shape[0] > 0):
				maxval = -9999
				[argmj, argmk] = [-1, -1]
				for j in range(0,tr_ind.shape[0]):		#model no
					for k in range(0,data_ind.shape[0]): 	#data track no
						#print([j, k])
						if(ll_matrix[j,k] > maxval and dd_matrix[j,k]>=dd_Threshold):
							maxval = ll_matrix[j,k]
							[argmj, argmk] = [j,k]
				if argmj !=-1 and argmk!=-1:
					kmax = data_ind[argmk] 
					jmax = tr_ind[argmj]
					if maxval>Threshold:
						#we got a match
						ls_pos[jmax] = data_loc[kmax] 
						ls_tracks[jmax].append(data_file[kmax])
						ls_labels[jmax].append(data_label[kmax])
						ls_feats[jmax].append(data_feat[kmax])
						ls_diff.append(jmax)
						ll_matrix = np.delete(ll_matrix, argmj, axis=0)
						ll_matrix = np.delete(ll_matrix, argmk, axis=1)
						data_ind = np.delete(data_ind, argmk,axis=0)
						tr_ind = np.delete(tr_ind, argmj,axis=0)
					else:
						#no match i.e. new track
						ls_pos.append(data_loc[kmax])
						ls_tracks.append([data_file[kmax]])
						ls_labels.append([data_label[kmax]])
						ls_feats.append([data_feat[kmax]])
						ls_diff.append(len(ls_tracks)-1)
						ll_matrix = np.delete(ll_matrix, argmk, axis=1)
						data_ind = np.delete(data_ind, argmk,axis=0)
				else:
					#no match i.e. new track
					ls_pos.append(data_loc[data_ind[0]])
					ls_tracks.append([data_file[data_ind[0]]])
					ls_labels.append([data_label[data_ind[0]]])
					ls_feats.append([data_feat[data_ind[0]]])
					ls_diff.append(len(ls_tracks)-1)
					data_ind = np.delete(data_ind, 0,axis=0)
			#sif len(ls_diff) > 1:
			np_tr_new = np.zeros((len(ls_tracks),len(ls_tracks)))
			np_tr_new[0:np_tr.shape[0],0:np_tr.shape[1]] = np_tr
			for t1 in range(len(ls_diff)):
				for t2 in range(t1+1, len(ls_diff)):
					np_tr_new[ls_diff[t1],ls_diff[t2]] += 1
					np_tr_new[ls_diff[t2],ls_diff[t1]] += 1
			np_tr = np_tr_new
			if j_c>=len(lsX):
				break
		ls_all.append(ls_tracks)
		ls_all_labels.append(ls_labels)
		ls_all_feats.append(ls_feats)
		ls_mats.append(np_tr)
		if j_c>=len(lsX):
			break
	return ls_all, ls_mats, ls_all_labels, ls_all_feats

In [11]:

def iterate_progressively(ls_all, ls_all_labels, ls_mats, ls_data):

		print("Iterate progressively.............")
		lsModels = []
		nModels = []
		labels_Models = []
        	#Initialize
		shotNo = -1
		trackNo = -1

		for shot, s_labels, rel_mat,data in itertools.izip(ls_all, ls_all_labels, ls_mats,ls_data):
			shotNo+=1
			shot_mat = np.greater(rel_mat, 0).astype(int)
			print("\nShot " + str(shotNo) + ": ")
			data = [np.array(t) for t in data]
			ll_matrix = np.zeros((len(lsModels),len(data)),dtype=float)
			yn_matrix = np.ones((len(lsModels),len(data)),dtype=float)
			#fill ll_matrix
			for j in range(0,len(lsModels)):		#model no
				for k in range(0,len(data)): 	#data track no
					sumVal=0
					for l in range(0, len(data[k])):
						sumVal+=np.sum((lsModels[j].means_[0] - data[k][l])**2)
					ll = sumVal/(1.0*len(data[k]))
					ll_matrix[j,k] = 4.0-ll
			data_ind = np.arange(len(data))
			#####################################################################################
			Threshold = 4.0-1.15
			#####################################################################################
			
			#the checking
		
			trackNo = 0
			np_occurence = np.zeros((len(data))).astype(int)
			while (data_ind.shape[0] > 0):
				trackNo+=1
				maxval = -9999
				[argmj, argmk] = [-1, -1]
				for j in range(0,len(lsModels)):		#model no
					for k in range(0,data_ind.shape[0]): 	#data track no
						if(yn_matrix[j,k]==1 and ll_matrix[j,k] > maxval):
							maxval = ll_matrix[j,k]
							[argmj, argmk] = [j,k]
				if argmj !=-1 and argmk!=-1:
					kmax = data_ind[argmk] 
					jmax = argmj 
					if maxval>Threshold:
						print("Data " + str(kmax) + " goes into model " + str(jmax))
						nModels[jmax], lsModels[jmax] = GMMu.update_GMM(lsModels[jmax], nModels[jmax], data[kmax])
						labels_Models[jmax] = labels_Models[jmax] + s_labels[kmax]
						np_occurence[kmax] = jmax
						yn_matrix[argmj,:] = yn_matrix[argmj,:] - shot_mat[argmk,:]
						for kko in range(data_ind.shape[0]):
							sumVal=0
							kkk = data_ind[kko]
							for lll in range(0, len(data[kkk])):
								sumVal+=np.sum((lsModels[jmax].means_[0] - data[kkk][lll])**2)
							ll = sumVal/(1.0*len(data[kkk]))
							ll_matrix[jmax,kko] = 4.0-ll
						

						ll_matrix = np.delete(ll_matrix, argmk, axis=1)
						yn_matrix = np.delete(yn_matrix, argmk, axis=1)
						shot_mat = np.delete(shot_mat, argmk,axis=0)
						shot_mat = np.delete(shot_mat, argmk,axis=1)
						data_ind = np.delete(data_ind, argmk,axis=0)
					else:
						print("Data " + str(kmax) + " is sent into new cluster.")
						n, l = GMMu.initialize_GMM(data[kmax])
						lsModels.append(l)
						nModels.append(n)
						labels_Models.append(s_labels[kmax])
						np_occurence[kmax] = len(lsModels) - 1

						if data_ind.shape[0] > 1:
							jjj = len(nModels) - 1
							yn_matrix = np.concatenate([yn_matrix, np.ones((1,data_ind.shape[0]))],axis=0)
							ll_matrix = np.concatenate([ll_matrix, np.zeros((1,data_ind.shape[0]))],axis=0)
							for kko in range(data_ind.shape[0]):
								sumVal=0
								kkk = data_ind[kko]
								for lll in range(0, len(data[kkk])):
									sumVal+=np.sum((lsModels[jjj].means_[0] - data[kkk][lll])**2)
								ll = sumVal/(1.0*len(data[kkk]))
								ll_matrix[jjj,kko] = 4.0-ll
						ll_matrix = np.delete(ll_matrix, argmk, axis=1)
						yn_matrix = np.delete(yn_matrix, argmk, axis=1)
						data_ind = np.delete(data_ind, argmk,axis=0)
						shot_mat = np.delete(shot_mat, argmk, axis=0)
						shot_mat = np.delete(shot_mat, argmk, axis=1)
				else:
						print("Data " + str(data_ind[0]) + " is initialised as a new cluster.")
						n, l = GMMu.initialize_GMM(data[data_ind[0]])
						lsModels.append(l)
						nModels.append(n)
						np_occurence[data_ind[0]] = len(lsModels) - 1
						labels_Models.append(s_labels[data_ind[0]])
						#update the ll matrix
						if data_ind.shape[0] > 1:
							jjj = len(nModels) - 1
							yn_matrix = np.concatenate([yn_matrix, np.ones((1,data_ind.shape[0]))],axis=0)
							yn_matrix[jjj,:] = yn_matrix[jjj,:] - shot_mat[0,:]
							ll_matrix = np.concatenate([ll_matrix, np.zeros((1,data_ind.shape[0]))],axis=0)
							for kko in range(data_ind.shape[0]):
								sumVal=0
								kkk = data_ind[kko]
								for lll in range(0, len(data[kkk])):
									sumVal+=np.sum((lsModels[jjj].means_[0] - data[kkk][lll])**2)
								ll = sumVal/(1.0*len(data[kkk]))
								ll = np.sum((sumVal/(1.0*len(data[kkk])))**2)
								ll_matrix[jjj,kko] = 4.0-ll
						ll_matrix = np.delete(ll_matrix, 0, axis=1)
						yn_matrix = np.delete(yn_matrix, 0, axis=1)
						data_ind = np.delete(data_ind, 0, axis=0)
						shot_mat = np.delete(shot_mat, 0, axis=0)
						shot_mat = np.delete(shot_mat, 0, axis=1)
		return nModels, labels_Models

In [12]:
def accuracy_score(y_act, y_pred):
	Cluster_Matrix = np.zeros((max(y_act)+1, max(y_pred)+1))
	print "Cluster_Matrix shape:"
	for i in range(len(y_act)):
		Cluster_Matrix[y_act[i], y_pred[i]]+=1.0	
	Inv_Cluster_Matrix = -1*Cluster_Matrix
	row, col = lsa(Inv_Cluster_Matrix)
	accu = np.sum(Cluster_Matrix[row, col])/float(np.sum(np.sum(Cluster_Matrix)))
	for i,j in itertools.izip(row,col):
		print(i,j,np.sum(Cluster_Matrix[i,:])/np.sum(np.sum(Cluster_Matrix)),Cluster_Matrix[i,j]/float(np.sum(Cluster_Matrix[:,j])))
	print(Cluster_Matrix.astype(int))
	print(row)
	print(col)
	return accu


def clustering_score(labels_Models):
	ls_allowed = [0,1,2,3,4,5,6,7]
	ls_pred = []
	for i in range(len(labels_Models)):
		for j in range(len(labels_Models[i])):
			ls_pred.append(i+1)
	ls_actual = []
	for item in labels_Models:
		ls_actual = ls_actual + item
	vm_score = vms(ls_actual, ls_pred)
	c_score = cs(ls_actual, ls_pred)
	h_score = hs(ls_actual, ls_pred)
	fm_score = fms(ls_actual, ls_pred)
	accu_score = accuracy_score(ls_actual, ls_pred)

	return vm_score, c_score, h_score, fm_score, accu_score

def check_track_uniformity(ls_labels):
	curr = 0
	tot = 0
	sid=0
	tid=0
	for ss in ls_labels:
		for t in ss:
			tot+=len(t)
			if len(np.unique(np.array(t))) > 1:
				print(sid,tid)
				print(np.unique(np.array(t)))
				print(t)
				print()
			else:
				curr+=len(t)
			tid+=1
		sid+=1
	print("Tracks purity: " + str(curr) + " / " + str(tot))

In [13]:
with open("./resources/ls_feats_list_nottinghill.pkl", 'rb' ) as fp:
    ls_feats_list = pkl.load(fp)
print(len(ls_feats_list))
print(ls_feats_list[0].shape)
ls_all, ls_mats, ls_all_labels, ls_data = get_vid_list(ls_feats_list)
del ls_feats_list
check_track_uniformity(ls_all_labels)
cnt_faces=0
for s in ls_all:
    for pll in s:
        cnt_faces+=len(pll)
print("Total faces available: " + str(cnt_faces))

19278
(128,)
No. of faces = 19278
ss range: [1678  125    0]
shot: 0, j_c: 0
shot: 1, j_c: 0
shot: 2, j_c: 0
shot: 3, j_c: 0
shot: 4, j_c: 0
shot: 5, j_c: 0
shot: 6, j_c: 0
shot: 7, j_c: 0
shot: 8, j_c: 0
shot: 9, j_c: 0
shot: 10, j_c: 0
shot: 11, j_c: 0
shot: 12, j_c: 0
shot: 13, j_c: 0
shot: 14, j_c: 0
shot: 15, j_c: 0
shot: 16, j_c: 0
shot: 17, j_c: 0
shot: 18, j_c: 0
shot: 19, j_c: 0
shot: 20, j_c: 0
shot: 21, j_c: 0
shot: 22, j_c: 0
shot: 23, j_c: 0
shot: 24, j_c: 0
shot: 25, j_c: 0
shot: 26, j_c: 0
shot: 27, j_c: 0
shot: 28, j_c: 0
shot: 29, j_c: 0
shot: 30, j_c: 0
shot: 31, j_c: 0
shot: 32, j_c: 0
shot: 33, j_c: 0
shot: 34, j_c: 0
shot: 35, j_c: 0
shot: 36, j_c: 0
shot: 37, j_c: 0
shot: 38, j_c: 0
shot: 39, j_c: 0
shot: 40, j_c: 0
shot: 41, j_c: 0
shot: 42, j_c: 0
shot: 43, j_c: 0
shot: 44, j_c: 0
shot: 45, j_c: 0
shot: 46, j_c: 0
shot: 47, j_c: 0
shot: 48, j_c: 0
shot: 49, j_c: 0
shot: 50, j_c: 0
shot: 51, j_c: 0
shot: 52, j_c: 0
shot: 53, j_c: 0
shot: 54, j_c: 0
shot: 55, j_c:

shot: 457, j_c: 2618
shot: 458, j_c: 2619
shot: 459, j_c: 2701
shot: 460, j_c: 2701
shot: 461, j_c: 2701
shot: 462, j_c: 2701
shot: 463, j_c: 2701
shot: 464, j_c: 2701
shot: 465, j_c: 2701
shot: 466, j_c: 2701
shot: 467, j_c: 2701
shot: 468, j_c: 2701
shot: 469, j_c: 2701
shot: 470, j_c: 2702
shot: 471, j_c: 2788
shot: 472, j_c: 2790
shot: 473, j_c: 2884
shot: 474, j_c: 2885
shot: 475, j_c: 2928
shot: 476, j_c: 2928
shot: 477, j_c: 2978
shot: 478, j_c: 3018
shot: 479, j_c: 3018
shot: 480, j_c: 3018
shot: 481, j_c: 3018
shot: 482, j_c: 3018
shot: 483, j_c: 3018
shot: 484, j_c: 3018
shot: 485, j_c: 3018
shot: 486, j_c: 3018
shot: 487, j_c: 3018
shot: 488, j_c: 3018
shot: 489, j_c: 3018
shot: 490, j_c: 3018
shot: 491, j_c: 3018
shot: 492, j_c: 3018
shot: 493, j_c: 3018
shot: 494, j_c: 3018
shot: 495, j_c: 3018
shot: 496, j_c: 3018
shot: 497, j_c: 3018
shot: 498, j_c: 3018
shot: 499, j_c: 3097
shot: 500, j_c: 3101
shot: 501, j_c: 3177
shot: 502, j_c: 3177
shot: 503, j_c: 3177
shot: 504, j_

shot: 962, j_c: 8478
shot: 963, j_c: 8478
shot: 964, j_c: 8478
shot: 965, j_c: 8478
shot: 966, j_c: 8478
shot: 967, j_c: 8478
shot: 968, j_c: 8478
shot: 969, j_c: 8478
shot: 970, j_c: 8478
shot: 971, j_c: 8478
shot: 972, j_c: 8478
shot: 973, j_c: 8478
shot: 974, j_c: 8478
shot: 975, j_c: 8478
shot: 976, j_c: 8478
shot: 977, j_c: 8478
shot: 978, j_c: 8478
shot: 979, j_c: 8478
shot: 980, j_c: 8478
shot: 981, j_c: 8478
shot: 982, j_c: 8478
shot: 983, j_c: 8478
shot: 984, j_c: 8478
shot: 985, j_c: 8478
shot: 986, j_c: 8478
shot: 987, j_c: 8478
shot: 988, j_c: 8478
shot: 989, j_c: 8478
shot: 990, j_c: 8478
shot: 991, j_c: 8478
shot: 992, j_c: 8478
shot: 993, j_c: 8478
shot: 994, j_c: 8478
shot: 995, j_c: 8519
shot: 996, j_c: 8519
shot: 997, j_c: 8588
shot: 998, j_c: 8588
shot: 999, j_c: 8588
shot: 1000, j_c: 8588
shot: 1001, j_c: 8588
shot: 1002, j_c: 8588
shot: 1003, j_c: 8588
shot: 1004, j_c: 8588
shot: 1005, j_c: 8588
shot: 1006, j_c: 8588
shot: 1007, j_c: 8588
shot: 1008, j_c: 8588
shot

shot: 1352, j_c: 14866
shot: 1353, j_c: 14866
shot: 1354, j_c: 14866
shot: 1355, j_c: 14866
shot: 1356, j_c: 14866
shot: 1357, j_c: 14866
shot: 1358, j_c: 14870
shot: 1359, j_c: 14974
shot: 1360, j_c: 14974
shot: 1361, j_c: 15089
shot: 1362, j_c: 15089
shot: 1363, j_c: 15089
shot: 1364, j_c: 15089
shot: 1365, j_c: 15089
shot: 1366, j_c: 15089
shot: 1367, j_c: 15093
shot: 1368, j_c: 15186
shot: 1369, j_c: 15186
shot: 1370, j_c: 15186
shot: 1371, j_c: 15186
shot: 1372, j_c: 15186
shot: 1373, j_c: 15186
shot: 1374, j_c: 15186
shot: 1375, j_c: 15186
shot: 1376, j_c: 15654
shot: 1377, j_c: 15654
shot: 1378, j_c: 15654
shot: 1379, j_c: 15654
shot: 1380, j_c: 15654
shot: 1381, j_c: 15654
shot: 1382, j_c: 15654
shot: 1383, j_c: 15654
shot: 1384, j_c: 15654
shot: 1385, j_c: 15654
shot: 1386, j_c: 15654
shot: 1387, j_c: 15654
shot: 1388, j_c: 15654
shot: 1389, j_c: 15654
shot: 1390, j_c: 15654
shot: 1391, j_c: 15654
shot: 1392, j_c: 15654
shot: 1393, j_c: 15654
shot: 1394, j_c: 15654
shot: 1395,

In [14]:
nModels, labels_Models = iterate_progressively(ls_all, ls_all_labels, ls_mats, ls_data)
vm_score, c_score, h_score, fm_score, accu_score = clustering_score(labels_Models)
print("Labels score: hom:" + str(h_score) + ", compl:" + str(c_score) + ", vm:" + str(vm_score) + ", fm:" + str(fm_score) + ", accu:" + str(accu_score))


Iterate progressively.............

Shot 0: 

Shot 1: 

Shot 2: 

Shot 3: 

Shot 4: 

Shot 5: 

Shot 6: 

Shot 7: 

Shot 8: 

Shot 9: 

Shot 10: 

Shot 11: 

Shot 12: 

Shot 13: 

Shot 14: 

Shot 15: 

Shot 16: 

Shot 17: 

Shot 18: 

Shot 19: 

Shot 20: 

Shot 21: 

Shot 22: 

Shot 23: 

Shot 24: 

Shot 25: 

Shot 26: 

Shot 27: 

Shot 28: 

Shot 29: 

Shot 30: 

Shot 31: 

Shot 32: 

Shot 33: 

Shot 34: 

Shot 35: 

Shot 36: 

Shot 37: 

Shot 38: 

Shot 39: 

Shot 40: 

Shot 41: 

Shot 42: 

Shot 43: 

Shot 44: 

Shot 45: 

Shot 46: 

Shot 47: 

Shot 48: 

Shot 49: 

Shot 50: 

Shot 51: 

Shot 52: 

Shot 53: 

Shot 54: 

Shot 55: 

Shot 56: 

Shot 57: 

Shot 58: 

Shot 59: 

Shot 60: 

Shot 61: 

Shot 62: 

Shot 63: 

Shot 64: 

Shot 65: 

Shot 66: 

Shot 67: 

Shot 68: 

Shot 69: 

Shot 70: 

Shot 71: 

Shot 72: 

Shot 73: 

Shot 74: 

Shot 75: 

Shot 76: 

Shot 77: 

Shot 78: 

Shot 79: 

Shot 80: 

Shot 81: 

Shot 82: 

Shot 83: 

Shot 84: 

Shot 85: 

Shot 86: 

Shot 87: 

Shot 8

Shot 433: 

Shot 434: 

Shot 435: 

Shot 436: 

Shot 437: 

Shot 438: 

Shot 439: 

Shot 440: 

Shot 441: 

Shot 442: 

Shot 443: 

Shot 444: 

Shot 445: 

Shot 446: 
Data 0 goes into model 0
d::128

Shot 447: 

Shot 448: 

Shot 449: 

Shot 450: 

Shot 451: 
Data 0 goes into model 0
d::128

Shot 452: 
Data 0 goes into model 0
d::128

Shot 453: 
Data 1 goes into model 0
d::128
Data 0 goes into model 3
d::128

Shot 454: 
Data 0 goes into model 0
d::128

Shot 455: 
Data 1 goes into model 0
d::128
Data 0 goes into model 3
d::128

Shot 456: 
Data 0 goes into model 0
d::128

Shot 457: 
Data 0 goes into model 0
d::128

Shot 458: 
Data 0 goes into model 0
d::128

Shot 459: 

Shot 460: 

Shot 461: 

Shot 462: 

Shot 463: 

Shot 464: 

Shot 465: 

Shot 466: 

Shot 467: 

Shot 468: 

Shot 469: 
Data 0 goes into model 0
d::128

Shot 470: 
Data 0 goes into model 0
d::128

Shot 471: 
Data 1 goes into model 0
d::128
Data 0 goes into model 3
d::128

Shot 472: 
Data 0 goes into model 0
d::128

Shot 473

Data 3 goes into model 5
d::128
Data 1 goes into model 3
d::128
Data 0 goes into model 0
d::128

Shot 594: 
Data 5 goes into model 4
d::128
Data 3 goes into model 4
d::128
Data 0 goes into model 4
d::128
Data 2 goes into model 5
d::128
Data 4 goes into model 5
d::128
Data 1 goes into model 5
d::128

Shot 595: 

Shot 596: 

Shot 597: 

Shot 598: 

Shot 599: 

Shot 600: 

Shot 601: 

Shot 602: 

Shot 603: 

Shot 604: 

Shot 605: 

Shot 606: 
Data 0 goes into model 4
d::128

Shot 607: 

Shot 608: 

Shot 609: 

Shot 610: 
Data 0 goes into model 4
d::128

Shot 611: 

Shot 612: 

Shot 613: 

Shot 614: 

Shot 615: 

Shot 616: 

Shot 617: 

Shot 618: 

Shot 619: 

Shot 620: 

Shot 621: 

Shot 622: 

Shot 623: 

Shot 624: 

Shot 625: 

Shot 626: 

Shot 627: 

Shot 628: 

Shot 629: 

Shot 630: 

Shot 631: 

Shot 632: 

Shot 633: 

Shot 634: 

Shot 635: 

Shot 636: 
Data 1 goes into model 4
d::128
Data 0 goes into model 1
d::128

Shot 637: 
Data 0 goes into model 4
d::128

Shot 638: 

Shot 639: 


Data 0 goes into model 0
d::128

Shot 888: 

Shot 889: 

Shot 890: 
Data 5 goes into model 3
d::128
Data 7 goes into model 3
d::128
Data 3 goes into model 3
d::128
Data 10 goes into model 3
d::128
Data 9 goes into model 3
d::128
Data 12 goes into model 3
d::128
Data 14 goes into model 3
d::128
Data 0 goes into model 3
d::128
Data 8 goes into model 0
d::128
Data 13 goes into model 0
d::128
Data 4 goes into model 0
d::128
Data 2 goes into model 0
d::128
Data 11 goes into model 0
d::128
Data 1 goes into model 0
d::128
Data 6 goes into model 0
d::128

Shot 891: 

Shot 892: 

Shot 893: 

Shot 894: 

Shot 895: 

Shot 896: 

Shot 897: 

Shot 898: 

Shot 899: 

Shot 900: 

Shot 901: 

Shot 902: 

Shot 903: 

Shot 904: 

Shot 905: 

Shot 906: 

Shot 907: 

Shot 908: 

Shot 909: 
Data 0 goes into model 3
d::128

Shot 910: 

Shot 911: 

Shot 912: 

Shot 913: 

Shot 914: 

Shot 915: 

Shot 916: 

Shot 917: 

Shot 918: 

Shot 919: 

Shot 920: 

Shot 921: 

Shot 922: 

Shot 923: 

Shot 924: 

Shot 9

d::128
Data 10 goes into model 1
d::128
Data 7 goes into model 1
d::128
Data 6 goes into model 1
d::128
Data 11 goes into model 0
d::128
Data 4 goes into model 0
d::128

Shot 1163: 
Data 1 goes into model 3
d::128
Data 0 goes into model 1
d::128
Data 2 goes into model 1
d::128

Shot 1164: 

Shot 1165: 

Shot 1166: 
Data 3 goes into model 1
d::128
Data 0 goes into model 1
d::128
Data 4 goes into model 3
d::128
Data 5 goes into model 1
d::128
Data 2 goes into model 3
d::128
Data 6 goes into model 3
d::128
Data 1 goes into model 3
d::128

Shot 1167: 

Shot 1168: 

Shot 1169: 
Data 1 goes into model 3
d::128
Data 0 goes into model 1
d::128

Shot 1170: 
Data 1 goes into model 1
d::128
Data 3 goes into model 1
d::128
Data 2 goes into model 3
d::128
Data 0 goes into model 3
d::128
Data 4 goes into model 1
d::128
Data 5 goes into model 0
d::128

Shot 1171: 

Shot 1172: 

Shot 1173: 

Shot 1174: 

Shot 1175: 

Shot 1176: 

Shot 1177: 

Shot 1178: 

Shot 1179: 
Data 4 goes into model 0
d::128
Da

Data 3 goes into model 4
d::128
Data 5 goes into model 1
d::128
Data 2 goes into model 1
d::128
Data 0 goes into model 1
d::128
Data 1 goes into model 4
d::128
Data 6 goes into model 3
d::128
Data 4 goes into model 3
d::128

Shot 1368: 

Shot 1369: 

Shot 1370: 

Shot 1371: 

Shot 1372: 

Shot 1373: 

Shot 1374: 

Shot 1375: 
Data 28 goes into model 4
d::128
Data 33 goes into model 4
d::128
Data 7 goes into model 3
d::128
Data 11 goes into model 3
d::128
Data 17 goes into model 3
d::128
Data 12 goes into model 3
d::128
Data 10 goes into model 3
d::128
Data 30 goes into model 1
d::128
Data 9 goes into model 3
d::128
Data 13 goes into model 3
d::128
Data 15 goes into model 3
d::128
Data 20 goes into model 3
d::128
Data 19 goes into model 3
d::128
Data 8 goes into model 3
d::128
Data 22 goes into model 4
d::128
Data 31 goes into model 4
d::128
Data 27 goes into model 1
d::128
Data 34 goes into model 1
d::128
Data 42 goes into model 1
d::128
Data 29 goes into model 1
d::128
Data 40 goes in

Data 16 goes into model 4
d::128
Data 56 goes into model 4
d::128
Data 3 goes into model 3
d::128
Data 55 goes into model 4
d::128
Data 67 goes into model 4
d::128
Data 28 goes into model 4
d::128
Data 21 goes into model 3
d::128
Data 61 goes into model 4
d::128
Data 71 goes into model 4
d::128
Data 20 goes into model 4
d::128
Data 0 goes into model 4
d::128
Data 35 goes into model 4
d::128
Data 15 goes into model 3
d::128
Data 5 goes into model 4
d::128
Data 33 goes into model 3
d::128
Data 70 goes into model 3
d::128
Data 52 goes into model 3
d::128
Data 23 goes into model 3
d::128
Data 63 goes into model 3
d::128
Data 27 goes into model 3
d::128
Data 18 goes into model 3
d::128
Data 72 goes into model 3
d::128
Data 54 goes into model 3
d::128
Data 49 goes into model 3
d::128
Data 36 goes into model 3
d::128
Data 29 goes into model 3
d::128
Data 38 goes into model 3
d::128
Data 50 goes into model 3
d::128
Data 62 goes into model 3
d::128
Data 42 goes into model 3
d::128
Data 40 goes 

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
